In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pathlib
pathlib.Path.ls = lambda x: list(x.iterdir())

In [3]:
import os
from datetime import datetime

import numpy as np
from torch.nn.utils import clip_grad_norm_
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

In [4]:
from evaluate import evaluate
from onsets_and_frames import *

In [5]:
logdir = 'runs/model'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
iterations = 500000 + 100
resume_iteration = 500000
checkpoint_interval = 1000
train_on = 'MAESTRO'

batch_size = 8
sequence_length = 327680
model_complexity = 48

if torch.cuda.is_available() and torch.cuda.get_device_properties(torch.cuda.current_device()).total_memory < 10e9:
    batch_size //= 2
    sequence_length //= 2
    print(f'Reducing batch size to {batch_size} and sequence_length to {sequence_length} to save memory')

learning_rate = 0.0006
learning_rate_decay_steps = 10000
learning_rate_decay_rate = 0.98

leave_one_out = None

clip_gradient_norm = 3

validation_length = sequence_length
validation_interval = 500


In [6]:
writer = SummaryWriter(logdir)

train_groups, validation_groups = ['train'], ['validation']

if leave_one_out is not None:
    all_years = {'2004', '2006', '2008', '2009', '2011', '2013', '2014', '2015', '2017'}
    train_groups = list(all_years - {str(leave_one_out)})
    validation_groups = [str(leave_one_out)]

if train_on == 'MAESTRO':
    dataset = MAESTRO(groups=train_groups, sequence_length=sequence_length)
    validation_dataset = MAESTRO(groups=validation_groups, sequence_length=sequence_length)
else:
    dataset = MAPS(groups=['AkPnBcht', 'AkPnBsdf', 'AkPnCGdD', 'AkPnStgb', 'SptkBGAm', 'SptkBGCl', 'StbgTGd2'], sequence_length=sequence_length)
    validation_dataset = MAPS(groups=['ENSTDkAm', 'ENSTDkCl'], sequence_length=validation_length)

Loading group train:   0%|          | 3/954 [00:00<00:51, 18.49it/s]

Loading 1 group of MAESTRO at data/MAESTRO


Loading group validation:   0%|          | 0/105 [00:00<?, ?it/s]

Loading 1 group of MAESTRO at data/MAESTRO


Loading group validation: 100%|██████████| 105/105 [01:41<00:00,  1.04it/s]


In [7]:
import torch
import torch.nn.functional as F
from torch import nn

from onsets_and_frames.lstm import BiLSTM
from onsets_and_frames.mel import melspectrogram
from onsets_and_frames.transcriber import ConvStack

In [8]:
class OnsetsAndFramesV2(nn.Module):
    def __init__(self, input_features, output_features, model_complexity=48):
        super().__init__()

        model_size = model_complexity * 16
        sequence_model = lambda input_size, output_size: BiLSTM(input_size, output_size // 2)

        self.onset_stack = nn.Sequential(
            ConvStack(input_features, model_size),
            sequence_model(model_size, model_size),
            nn.Linear(model_size, output_features),
            nn.Sigmoid()
        )
        self.offset_stack = nn.Sequential(
            ConvStack(input_features, model_size),
            sequence_model(model_size, model_size),
            nn.Linear(model_size, output_features),
            nn.Sigmoid()
        )
        self.frame_stack = nn.Sequential(
            ConvStack(input_features, model_size),
            nn.Linear(model_size, output_features),
            nn.Sigmoid()
        )
        #self.combined_stack = nn.Sequential(
        #    sequence_model(output_features * 3, model_size),
        #    nn.Linear(model_size, output_features),
        #    nn.Sigmoid()
        #)
        self.combined_stack1 = nn.Sequential(
            sequence_model(output_features * 3, model_size),
        )
        self.combined_stack2= nn.Sequential(
            nn.Linear(model_size, output_features),
            nn.Sigmoid()
        )
        self.shift_param_stack = nn.Sequential(
            nn.Linear(model_size * 2, 1)
        )
        self.velocity_stack = nn.Sequential(
            ConvStack(input_features, model_size),
            nn.Linear(model_size, output_features)
        )
        
        
    def forward(self, mel):
        onset_pred = self.onset_stack(mel)
        offset_pred = self.offset_stack(mel)
        activation_pred = self.frame_stack(mel)
        combined_pred = torch.cat([onset_pred.detach(), offset_pred.detach(), activation_pred], dim=-1)

        #frame_pred = self.combined_stack(combined_pred)
        co_used_pred = self.combined_stack1(combined_pred)
        frame_pred = self.combined_stack2(co_used_pred)
        
        velocity_pred = self.velocity_stack(mel)
        return onset_pred, offset_pred, activation_pred, frame_pred, velocity_pred, co_used_pred

    def SetFreeze(self, is_freeze, is_co_use_freeze, is_frame_freeze, is_shift_freeze):
        for p in self.parameters():
            p.requires_grad = not is_freeze
            
        for p in self.combined_stack1.parameters():
            p.requires_grad = not is_co_use_freeze

        for p in self.combined_stack2.parameters():
            p.requires_grad = not is_frame_freeze
            
        for p in self.shift_param_stack.parameters():
            p.requires_grad = not is_shift_freeze
        
    def run_shift_param_on_batch(self, batch):
        audio_label = batch['audio']
        onset_label = batch['onset']
        offset_label = batch['offset']
        frame_label = batch['frame']
        velocity_label = batch['velocity']
        
        shift_audio_label = batch['shift_audio']
        shift_label = batch['shift']

        mel = melspectrogram(audio_label.reshape(-1, audio_label.shape[-1])[:, :-1]).transpose(-1, -2)
        onset_pred, offset_pred, _, frame_pred, velocity_pred, co_used_pred = self(mel)

        mel_shift = melspectrogram(shift_audio_label.reshape(-1, shift_audio_label.shape[-1])[:, :-1]).transpose(-1, -2)
        onset_shift_pred, offset_shift_pred, _, frame_shift_pred, velocity_shift_pred, co_used_shift_pred = self(mel_shift)

        two_co_used_pred = torch.cat([co_used_pred, co_used_shift_pred], dim=-1)
        shift_pred = self.shift_param_stack(two_co_used_pred).squeeze(-1)
        
        predictions = {
            'onset': onset_pred.reshape(*onset_label.shape),
            'offset': offset_pred.reshape(*offset_label.shape),
            'frame': frame_pred.reshape(*frame_label.shape),
            'velocity': velocity_pred.reshape(*velocity_label.shape),
            'shift': shift_pred.reshape(*shift_label.shape)
        }

        shift_loss = nn.MSELoss()
        losses = {
            'loss/onset': F.binary_cross_entropy(predictions['onset'], onset_label),
            'loss/offset': F.binary_cross_entropy(predictions['offset'], offset_label),
            'loss/frame': F.binary_cross_entropy(predictions['frame'], frame_label),
            'loss/velocity': self.velocity_loss(predictions['velocity'], velocity_label, onset_label),
            'loss/shift': F.mse_loss(predictions['shift'], shift_label)
        }

        return predictions, losses

    def run_on_batch(self, batch):
        audio_label = batch['audio']
        onset_label = batch['onset']
        offset_label = batch['offset']
        frame_label = batch['frame']
        velocity_label = batch['velocity']

        mel = melspectrogram(audio_label.reshape(-1, audio_label.shape[-1])[:, :-1]).transpose(-1, -2)
        onset_pred, offset_pred, _, frame_pred, velocity_pred, _ = self(mel)

        predictions = {
            'onset': onset_pred.reshape(*onset_label.shape),
            'offset': offset_pred.reshape(*offset_label.shape),
            'frame': frame_pred.reshape(*frame_label.shape),
            'velocity': velocity_pred.reshape(*velocity_label.shape)
        }

        losses = {
            'loss/onset': F.binary_cross_entropy(predictions['onset'], onset_label),
            'loss/offset': F.binary_cross_entropy(predictions['offset'], offset_label),
            'loss/frame': F.binary_cross_entropy(predictions['frame'], frame_label),
            'loss/velocity': self.velocity_loss(predictions['velocity'], velocity_label, onset_label)
        }

        return predictions, losses
    
    def velocity_loss(self, velocity_pred, velocity_label, onset_label):
        denominator = onset_label.sum()
        if denominator.item() == 0:
            return denominator
        else:
            return (onset_label * (velocity_label - velocity_pred) ** 2).sum() / denominator


In [9]:
from collections import OrderedDict
def rename_key(old_dict, old_key, new_key):
    return OrderedDict((new_key if k == old_key else k, v) for k, v in old_dict.items())    

def load_and_adapt_model(model_path, device):
    old_model = torch.load(model_path)

    model_state = old_model.state_dict()

    model_state = rename_key(model_state,'combined_stack.0.rnn.weight_ih_l0', 'combined_stack1.0.rnn.weight_ih_l0')
    model_state = rename_key(model_state,'combined_stack.0.rnn.weight_hh_l0', 'combined_stack1.0.rnn.weight_hh_l0')
    model_state = rename_key(model_state,'combined_stack.0.rnn.bias_ih_l0', 'combined_stack1.0.rnn.bias_ih_l0')
    model_state = rename_key(model_state,'combined_stack.0.rnn.bias_hh_l0', 'combined_stack1.0.rnn.bias_hh_l0')

    model_state = rename_key(model_state,'combined_stack.0.rnn.weight_ih_l0_reverse', 'combined_stack1.0.rnn.weight_ih_l0_reverse')
    model_state = rename_key(model_state,'combined_stack.0.rnn.weight_hh_l0_reverse', 'combined_stack1.0.rnn.weight_hh_l0_reverse')
    model_state = rename_key(model_state,'combined_stack.0.rnn.bias_ih_l0_reverse', 'combined_stack1.0.rnn.bias_ih_l0_reverse')
    model_state = rename_key(model_state,'combined_stack.0.rnn.bias_hh_l0_reverse', 'combined_stack1.0.rnn.bias_hh_l0_reverse')

    model_state = rename_key(model_state,'combined_stack.1.weight', 'combined_stack2.0.weight')
    model_state = rename_key(model_state,'combined_stack.1.bias', 'combined_stack2.0.bias')
    
    model = OnsetsAndFramesV2(N_MELS, MAX_MIDI - MIN_MIDI + 1, model_complexity)
    model.load_state_dict(model_state, strict=False)

    model = model.to(device)
    
    return model

In [10]:
model_path = "./runs/model/model-500000.pt"
model = load_and_adapt_model(model_path, device)

/home/superaoi/.local/share/virtualenvs/env-JrPVfsOJ/lib/python3.6/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/superaoi/.local/share/virtualenvs/env-JrPVfsOJ/lib/python3.6/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.modules.activation.ReLU' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/superaoi/.local/share/virtualenvs/env-JrPVfsOJ/lib/python3.6/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.modules.l

In [11]:
def init_default(m):
    "Initialize `m` weights with `func` and set `bias` to 0."
    if hasattr(m, 'weight'):
        nn.init.uniform_(m.weight)
    if hasattr(m, 'bias') and hasattr(m.bias, 'data'):
        m.bias.data.fill_(0.)
    return m


In [ ]:
##### NOTE: be careful to init

In [12]:
#init_default(model.shift_param_stack[0])

Linear(in_features=1536, out_features=1, bias=True)

In [13]:
batch_size=16
loader = DataLoader(dataset, batch_size, shuffle=True, drop_last=True)

In [14]:
model.SetFreeze(True, False, False, False)
model.train()

OnsetsAndFramesV2(
  (onset_stack): Sequential(
    (0): ConvStack(
      (cnn): Sequential(
        (0): Conv2d(1, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU()
        (6): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
        (7): Dropout(p=0.25, inplace=False)
        (8): Conv2d(48, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (9): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (10): ReLU()
        (11): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
        (12): Dropout(p=0.25, inplace=False)
      )
      (fc): Sequential(
        (0): 

In [15]:
optimizer = torch.optim.Adam(model.parameters(), learning_rate)
scheduler = StepLR(optimizer, step_size=learning_rate_decay_steps, gamma=learning_rate_decay_rate)

In [16]:
resume_iteration=500000
iterations=500000+500000
validation_interval = 500   #orig: 500
ouptut_interval=500

In [17]:
train_shift_loss = []
train_frame_loss = []

loop = tqdm(range(resume_iteration + 1, iterations + 1))
for i, batch in zip(loop, cycle(loader)):
    #predictions, losses = model.run_on_batch(batch)
    #loss = sum(losses.values())

    predictions_with_shift, losses_with_shift = model.run_shift_param_on_batch(batch)
    loss_shift = losses_with_shift['loss/shift']
    train_shift_loss.append(loss_shift.item())
    
    loss_frame = losses_with_shift['loss/frame']
    train_frame_loss.append(loss_frame.item())
    
    loss = loss_shift+loss_frame

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    scheduler.step()
    
    if (len(train_shift_loss)>=ouptut_interval):
        mean_train_shift_loss = np.mean(np.array(train_shift_loss))
        mean_train_frame_loss = np.mean(np.array(train_frame_loss))
        print (f"mean train loss: shift = {mean_train_shift_loss}, frame = {mean_train_frame_loss}")    
        train_shift_loss=[]
        train_frame_loss=[]

    if clip_gradient_norm:
        clip_grad_norm_(model.parameters(), clip_gradient_norm)

    for key, value in {'loss': loss, **losses_with_shift}.items():
        writer.add_scalar(key, value.item(), global_step=i)

    if i % validation_interval == 0:
        model.eval()
        with torch.no_grad():
            for key, value in evaluate(validation_dataset, model).items():
                writer.add_scalar('validation/' + key.replace(' ', '_'), np.mean(value), global_step=i)
        model.train()

    if i % checkpoint_interval == 0:
        torch.save(model, os.path.join(logdir, f'model-{i}.pt'))
        torch.save(optimizer.state_dict(), os.path.join(logdir, 'last-optimizer-state.pt'))


  0%|          | 499/500000 [05:03<84:48:46,  1.64it/s]

mean train loss: shift = 5.8234214532375335, frame = 0.04142925556376576


/home/superaoi/.local/share/virtualenvs/env-JrPVfsOJ/lib/python3.6/site-packages/mir_eval/transcription_velocity.py:188: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  ref_matched_velocities)[0]
  0%|          | 999/500000 [10:34<84:49:14,  1.63it/s]  

mean train loss: shift = 1.8265601613521576, frame = 0.06011466918885708


/home/superaoi/.local/share/virtualenvs/env-JrPVfsOJ/lib/python3.6/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type OnsetsAndFramesV2. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
  0%|          | 1499/500000 [16:03<84:40:27,  1.64it/s]  

...

100%|█████████▉| 499499/500000 [92:10:31<05:05,  1.64it/s]  

mean train loss: shift = 0.021778937647119165, frame = 0.02065453510172665


100%|█████████▉| 499999/500000 [92:16:04<00:00,  1.64it/s]  

mean train loss: shift = 0.01949497579038143, frame = 0.020870717661455273


100%|██████████| 500000/500000 [92:16:32<00:00,  1.51it/s]


In [ ]:
    if resume_iteration is None:
        model = OnsetsAndFrames(N_MELS, MAX_MIDI - MIN_MIDI + 1, model_complexity).to(device)
        optimizer = torch.optim.Adam(model.parameters(), learning_rate)
        resume_iteration = 0
    else:
        model_path = os.path.join(logdir, f'model-{resume_iteration}.pt')
        model = torch.load(model_path)
        optimizer = torch.optim.Adam(model.parameters(), learning_rate)
        optimizer.load_state_dict(torch.load(os.path.join(logdir, 'last-optimizer-state.pt')))



In [33]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

leave_one_out=None
clip_gradient_norm = 3
batch_size = 2
sequence_length = 327680
model_complexity = 48


if torch.cuda.is_available() and torch.cuda.get_device_properties(torch.cuda.current_device()).total_memory < 10e9:
    batch_size //= 2
    sequence_length //= 2
    print(f'Reducing batch size to {batch_size} and sequence_length to {sequence_length} to save memory')

train_groups, validation_groups = ['train'], ['validation']

if leave_one_out is not None:
    all_years = {'2004', '2006', '2008', '2009', '2011', '2013', '2014', '2015', '2017'}
    train_groups = list(all_years - {str(leave_one_out)})
    validation_groups = [str(leave_one_out)]
    

In [6]:
dataset = MAESTRO(groups=train_groups, sequence_length=sequence_length)

Loading group train:   0%|          | 0/954 [00:00<?, ?it/s]

Loading 1 group of MAESTRO at data/MAESTRO


Loading group train: 100%|██████████| 954/954 [01:12<00:00, 13.22it/s]

load 8 data. early return for debug purpose
load 8 data. early return for debug purpose
load 8 data. early return for debug purpose
load 8 data. early return for debug purpose
load 8 data. early return for debug purpose
load 8 data. early return for debug purpose
load 8 data. early return for debug purpose
load 8 data. early return for debug purpose
load 8 data. early return for debug purpose
load 8 data. early return for debug purpose
load 8 data. early return for debug purpose
load 8 data. early return for debug purpose
load 8 data. early return for debug purpose
load 8 data. early return for debug purpose
load 8 data. early return for debug purpose
load 8 data. early return for debug purpose
load 8 data. early return for debug purpose
load 8 data. early return for debug purpose
load 8 data. early return for debug purpose
load 8 data. early return for debug purpose
load 8 data. early return for debug purpose
load 8 data. early return for debug purpose
load 8 data. early return for de

### try to debug run_shift_param_on_batch

In [58]:
logdir="runs/model"
writer = SummaryWriter(logdir)
train_groups, validation_groups = ['train'], ['validation']

if leave_one_out is not None:
    all_years = {'2004', '2006', '2008', '2009', '2011', '2013', '2014', '2015', '2017'}
    train_groups = list(all_years - {str(leave_one_out)})
    validation_groups = [str(leave_one_out)]

if train_on == 'MAESTRO':
    dataset = MAESTRO(groups=train_groups, sequence_length=sequence_length)
    validation_dataset = MAESTRO(groups=validation_groups, sequence_length=sequence_length)
else:
    dataset = MAPS(groups=['AkPnBcht', 'AkPnBsdf', 'AkPnCGdD', 'AkPnStgb', 'SptkBGAm', 'SptkBGCl', 'StbgTGd2'], sequence_length=sequence_length)
    validation_dataset = MAPS(groups=['ENSTDkAm', 'ENSTDkCl'], sequence_length=validation_length)
    

In [20]:
onebatch = next(iter(loader))

In [21]:
with torch.no_grad():
    pred, lls = model.run_shift_param_on_batch(onebatch)

In [30]:
print (lls['loss/onset'].item())

0.0033941396977752447


In [23]:
print (pred["shift"])

tensor([[ 0.0197, -0.0122,  0.1601,  ..., -2.7126, -0.8132, -0.6011],
        [ 1.6764,  2.0073,  2.0166,  ..., -0.6118,  0.0727,  0.1484],
        [ 1.8172,  1.7645,  1.5668,  ...,  0.6216,  0.6121,  0.3840],
        ...,
        [-0.6990, -0.2199, -0.5687,  ..., -0.3635, -0.2629,  0.2758],
        [-0.4857,  0.4936, -1.2555,  ...,  0.3769,  0.1807,  0.2952],
        [ 1.2530,  1.6878,  1.8124,  ...,  1.4559,  1.7927,  2.1411]],
       device='cuda:0')


In [22]:
print (onebatch['shift'])
#print (pred['shift'][0,:])

tensor([[-1., -1., -1.,  ..., -1., -1., -1.],
        [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
        [ 3.,  3.,  3.,  ...,  3.,  3.,  3.],
        ...,
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [ 4.,  4.,  4.,  ...,  4.,  4.,  4.]], device='cuda:0')


In [15]:
predictions, losses = new_model.run_on_batch(batch)


In [46]:
aaa = torch.randn(2,10)
bbb = torch.randn(2,10)
aaa *= 0
bbb[0,:] = 0.5
bbb[1,:] = 0.0
print (aaa)
print (bbb)
F.mse_loss(aaa, bbb)

tensor([[-0., 0., 0., 0., 0., 0., 0., 0., -0., 0.],
        [-0., -0., -0., 0., 0., -0., 0., -0., 0., -0.]])
tensor([[0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
         0.5000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000]])


tensor(0.1250)

In [47]:
0.5*0.5*10/20

0.125

In [16]:
print (losses[])

{'loss/onset': tensor(0.0034, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>), 'loss/offset': tensor(0.0022, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>), 'loss/frame': tensor(0.0282, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>), 'loss/velocity': tensor(0.0048, device='cuda:0', grad_fn=<DivBackward0>)}


In [37]:
one = dataset[0]

In [40]:
print (one['audio'].shape, one['audio'].dtype)
print (one['shift_audio'].shape, one['shift_audio'].dtype)
print (one['label'].shape)
print (one['shift'].shape, one['shift'].dtype)
a = one['shift'].float()
print (a.shape, a.dtype)

torch.Size([327680]) torch.float32
torch.Size([327680]) torch.float32
torch.Size([640, 88])
torch.Size([640]) torch.uint8
torch.Size([640]) torch.float32


In [39]:
print (batch['audio'].shape)
print (batch['shift_audio'].shape)
print (batch['shift'].shape)

torch.Size([8, 327680])
torch.Size([8, 327680])
torch.Size([8, 640])


In [ ]:

def bn_drop_lin(n_in, n_out, bn=True, p:float=0., actn=False):
    "Sequence of batchnorm (if `bn`), dropout (with `p`) and linear (`n_in`,`n_out`) layers followed by `actn`."
    layers = [nn.BatchNorm1d(n_in)] if bn else []
    if p != 0: layers.append(nn.Dropout(p))
    layers.append(nn.Linear(n_in, n_out))
    if actn: layers.append(nn.ReLU(inplace=True))
    return layers


In [ ]:
class RegressionModel(nn.Module):
    def __init__(self, input_features):
        #input_features = model_size = 768
        super().__init__()

        layers = []
        layers += bn_drop_lin(input_features, 256, bn=True, p=0, actn=True)
        layers += bn_drop_lin(256, 64, bn=True, p=0, actn=True)
        layers += bn_drop_lin(64, 1, bn=False, p=0, actn=False)
        self.layers = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.layers(x)
    

In [70]:
reg = RegressionModel(768)

In [71]:
x = torch.randn((24, 768))
y=reg(x)
print (y.shape)

torch.Size([24, 1])


In [74]:
y_gt = torch.randn(24)
print (y_gt.shape)
print (y_gt)

torch.Size([24])
tensor([ 0.0791,  0.1717, -1.5212, -2.2675,  1.8543, -1.9075,  1.5579, -0.1735,
        -0.2960,  1.3799, -0.6627, -1.0970, -0.7396,  0.3816, -1.2872, -1.1826,
        -1.0749,  0.3559, -0.7001,  1.8105,  1.1595,  0.8026, -0.8735, -0.7194])


In [17]:
train_path = pathlib.Path('./data/MAESTRO')
train_path.ls()

[PosixPath('data/MAESTRO/2004'),
 PosixPath('data/MAESTRO/2009'),
 PosixPath('data/MAESTRO/LICENSE'),
 PosixPath('data/MAESTRO/2008'),
 PosixPath('data/MAESTRO/2017'),
 PosixPath('data/MAESTRO/2013'),
 PosixPath('data/MAESTRO/2006'),
 PosixPath('data/MAESTRO/maestro-v1.0.0.csv'),
 PosixPath('data/MAESTRO/2011'),
 PosixPath('data/MAESTRO/2014'),
 PosixPath('data/MAESTRO/2015'),
 PosixPath('data/MAESTRO/README'),
 PosixPath('data/MAESTRO/maestro-v1.0.0.json')]

In [ ]:
check_load = torch.load("./data/MAESTRO/2004/MIDI-Unprocessed_SMF_05_R1_2004_02-03_ORIG_MID--AUDIO_05_R1_2004_06_Track06_wav.pt")
for k in check_load.keys():
    print (k)

In [ ]:
model